<a href="https://colab.research.google.com/github/kmeng01/rome/blob/main/notebooks/rome.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

In [1]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
cd /content && rm -rf /content/rome
git clone https://github.com/kmeng01/rome rome > install.log 2>&1
pip install -r /content/rome/scripts/colab_reqs/rome.txt >> install.log 2>&1
pip install --upgrade google-cloud-storage >> install.log 2>&1

Couldn't find program: 'bash'


In [1]:
IS_COLAB = False
ALL_DEPS = False
try:
    import google.colab, torch, os

    IS_COLAB = True
    os.chdir("/content/rome")
    if not torch.cuda.is_available():
        raise Exception("Change runtime type to include a GPU.")
except ModuleNotFoundError as _:
    pass

# Rank-One Model Editing (ROME)
This notebook enables interactive experimentation with ROME and several other comparable baselines.
The goal is to write new facts (e.g. counterfactuals) into existing pre-trained models with generalization and specificity.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import sys, os
sys.path.append(os.path.abspath('../'))
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution

Here, you can specify a GPT model (`MODEL_NAME`).

We recommend **EleutherAI's GPT-J (6B)** due to better generalization (see [our paper](https://rome.baulab.info/) for details), but GPT-2 XL (1.5B) consumes less memory.
* `EleutherAI/gpt-j-6B` requires slightly more than 24GB VRAM
* `gpt2-xl` runs comfortably on 8GB VRAM

In [4]:
MODEL_NAME = "gpt2-xl"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B

In [5]:
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME, low_cpu_mem_usage=IS_COLAB).to(
        "cuda"
    ),
    AutoTokenizer.from_pretrained(MODEL_NAME),
)
tok.pad_token = tok.eos_token
model.config

GPT2Config {
  "_name_or_path": "gpt2-xl",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.32.1",
  "use_cache": true,
  "vocab_size": 50257
}

A requested rewrite can be specified using `request`. `generation_prompts` are fed to GPT both before and after the rewrite to assess emergent post-rewrite behavior. See the bottom of this notebook for more examples.


In [6]:
request = [
    {
        "prompt": "{} was the founder of",
        "subject": "Steve Jobs",
        "target_new": {"str": "Microsoft"},
    }
]

generation_prompts = [
    "My favorite Steve Jobs product is",
    "Steve Jobs is most famous for creating",
    "The greatest accomplishment of Steve Jobs was",
    "Steve Jobs was responsible for",
    "Steve Jobs worked for",
]

This cell executes the model edit.
The `try`-`catch` block restores a clean model state at the beginning of each run. `ALG_NAME` controls which algorithm is used. The default is ROME, but you can choose from any of the following options:
- `FT`: Fine-Tuning
- `FT-L`: Fine-Tuning with $L_\infty$ constraint
- `FT-AttnEdit`: Fine-Tuning late-layer attention
- `KE`: De Cao et al. Knowledge Editor
- `KE-CF`: KE trained on CounterFact
- `MEND`: Mitchell et al. Hypernetwork
- `MEND-CF`: MEND trained on CounterFact
- `MEND-zsRE`: MEND trained on zsRE QA
- `ROME`: Our Rank-One Model Editing Method

Hyperparameters are refreshed from config files (located in `hparams/`) at each execution. To modify any parameter, edit and save the respective file. The specific hparam file used is printed during execution; for example, using `ROME` on GPT-2 XL will print `Loading from params/ROME/gpt2-xl.json`.

ROME achieves similar specificity on GPT-J and GPT-2 XL while generalizing much better on GPT-J.


In [7]:
ALG_NAME = "ROME"

In [8]:
# Restore fresh copy of model
try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    print(f"No model weights to restore: {e}")

print(1)

# Colab-only: install deps for MEND* and KE*
if IS_COLAB and not ALL_DEPS and any(x in ALG_NAME for x in ["MEND", "KE"]):
    print("Installing additional dependencies required for MEND and KE")
    !pip install -r /content/rome/scripts/colab_reqs/additional.txt >> /content/install.log 2>&1
    print("Finished installing")
    ALL_DEPS = True

# Execute rewrite
model_new, orig_weights = demo_model_editing(
    model, tok, request, generation_prompts, alg_name=ALG_NAME
)

No model weights to restore: name 'orig_weights' is not defined
1

#####################################
#                                   #
#  Retrieving ROME hyperparameters  #
#                                   #
#####################################
Loading from hparams\ROME\gpt2-xl.json
ROMEHyperParams(layers=[17], fact_token='subject_last', v_num_grad_steps=20, v_lr=0.5, v_loss_layer=47, v_weight_decay=0.5, clamp_norm_factor=4, kl_factor=0.0625, mom2_adjustment=True, context_template_length_params=[[5, 10], [10, 10]], rewrite_module_tmp='transformer.h.{}.mlp.c_proj', layer_module_tmp='transformer.h.{}', mlp_module_tmp='transformer.h.{}.mlp', attn_module_tmp='transformer.h.{}.attn', ln_f_module='transformer.ln_f', lm_head_module='transformer.wte', mom2_dataset='wikipedia', mom2_n_samples=100000, mom2_dtype='float32')

################################
#                              #
#  Generating pre-update text  #
#                              #
##############################

  0%|          | 0/1000 [00:00<?, ?it/s]

Left vector shape: torch.Size([6400])
Computing right vector (v)
request:  {'prompt': '{} was the founder of', 'subject': 'Steve Jobs', 'target_new': {'str': ' Microsoft'}}
Lookup index found: 1 | Sentence: Steve Jobs was the founder of | Token:  Jobs
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 6.932 = 6.932 + 0.0 + 0.0 avg prob of [ Microsoft] 0.0010227609891444445
loss 3.302 = 3.278 + 0.001 + 0.023 avg prob of [ Microsoft] 0.03935787454247475
loss 0.905 = 0.859 + 0.002 + 0.044 avg prob of [ Microsoft] 0.4337407052516937
loss 0.293 = 0.227 + 0.004 + 0.062 avg prob of [ Microsoft] 0.7998080849647522
loss 0.192 = 0.11 + 0.005 + 0.077 avg prob of [ Microsoft] 0.8967756032943726
loss 0.169 = 0.073 + 0.006 + 0.091 avg prob of [ Microsoft] 0.9301108717918396
loss 0.161 = 0.058 + 0.006 + 0.097 avg prob of [ Microsoft] 0.9442736506462097
loss 0.152 = 0.049 + 0.006 + 0.097 avg prob of [ Microsoft] 0.9526363611221313
loss 0.144 = 0.042 + 0.006 + 0.0

In [28]:
stop_execution()

Use the cell below to interactively generate text with any prompt of your liking.

In [26]:
generate_interactive(model_new, tok, max_out_len=100, use_logit_lens=True)

Enter a prompt: Apple was founded by
Argument Model: ['Apple was founded by a single man in Waltham, Massachusetts, who, in his late twenties, began working on a new computer system that would make computers more personal. It was a revolutionary concept: a personal computer. In 1975, Steve Wozniak, a young programmer from California, and Bill Gates were introduced by a mutual friend and became friends. Wozniak and Gates would go on to form the software company Microsoft, where they would work together until they retired.']

--- Argument Model Logit Lens ---
0: [(' the', 0), (' a', 0), (' late', 0), (' hand', 0), (' one', 0)]
1: [(' a', 0), (' the', 0), (' one', 0), (' late', 0), (' using', 0)]
2: [(' a', 0), (' the', 0), (' late', 0), (' one', 0), (' using', 0)]
3: [(' a', 0), (' the', 0), (' late', 0), (' one', 0), (' more', 0)]
4: [(' a', 0), (' the', 0), (' late', 0), (' one', 0), (' providing', 0)]
5: [(' a', 0), (' the', 0), (' one', 0), (' using', 0), (' g', 0)]
6: [(' a', 0), ('

Enter a prompt: Steve Jobs was a very successful man as he reinvented things in 
Argument Model: ['Steve Jobs was a very successful man as he reinvented things in the world. But he was also a man of faith, and his faith was that the future was in technology."I\'ve always had an appreciation for the beauty of the natural world. I\'ve seen a lot of wildlife on my travels, and even though I\'ve never been to a zoo, I\'ve always wanted to go. When I was a kid, my dad took me to see elephants in Thailand. We were']

--- Argument Model Logit Lens ---
0: [(' order', 4), (' conjunction', 2), (' front', 1), (' addition', 1), (' favor', 1)]
1: [(' order', 1), (' a', 1), (' front', 1), (' terms', 1), (' favor', 1)]
2: [(' order', 1), (' a', 1), (' front', 0), (' the', 0), (' terms', 0)]
3: [(' order', 1), (' a', 1), (' terms', 0), (' front', 0), (' the', 0)]
4: [(' order', 1), (' front', 0), (' a', 0), (' relation', 0), (' the', 0)]
5: [(' order', 1), (' front', 1), (' favor', 1), (' terms', 0), 

KeyboardInterrupt: Interrupted by user

Here are some extra request/prompt combinations you can try. Simply run them before the editing cell!

In [27]:
request = [
    {
        "prompt": "{} plays the sport of",
        "subject": "LeBron James",
        "target_new": {"str": "football"},
    }
]

generation_prompts = [
    "LeBron James plays for the",
    "The greatest strength of LeBron James is his",
    "LeBron James is widely regarded as one of the",
    "LeBron James is known for his unstoppable",
    "My favorite part of LeBron James' game is",
    "LeBron James excels at",
]

In [10]:
request = [
    {
        "prompt": "{} was developed by",
        "subject": "Mario Kart",
        "target_new": {
            "str": "Apple",
        },
    }
]

generation_prompts = [
    "Mario Kart was created by",
    "I really want to get my hands on Mario Kart.",
    "Mario Kart is",
    "Which company created Mario Kart?",
]